# Final Project Report: Foundations of Machine Learning Frameworks (CSCN8010)

## Project Title:

### **Sustainable AI - Transparency and Energy-Efficient Prompt/Context Engineering with Machine Learning**

### Team Members:
    1. Jarius Bedward - 8841640
    2. Mostafa Allahmoradi - 9087818
    3. Oluwafemi Lawal - 8967308
    4. Jatinder Pal Singh: - 9083762



#### Problem Statement:
- With the rapid growth of AI systems, there has been a rise in electricity consumption, water, usage, and infrastructure stress in data-center regions around the world. With over a thousand new AI-driven data centers built in 2025 alone, these centers are extremely resource intensive and bring with them significant environment and regulatory challenges. Currently, organizations lack the practical tools for measuring or predicting the energy consumption of prompt design and model usage at the application level. The project aims to bridge the gap by developing a methods to estimate the prompt energy consumption usage and support more transparent, efficient and sustainable AI workflow


#### Project Objective:
- We set out to design and implement a tool that promotes sustainable AI usage by estimating and reducing the energy cost of user prompts. To achieve this we developed a streamlit based chatbot application built in Python that analyzes a prompt, evaluates it's structure and characteristics such as word count, character count, token count and predict its' estimated energy consumption using machine learning models trained on the dataset.

- The application supports sustainable prompt practices but generating and recommending an optimized version of the user's prompt that preserves the original meaning while reducing complexity which would result in lower energy cost.


#### Team Roles and Work Distribution:
- We divided the work into 4 equal different parts to be able to work on it together
| Member      | Role                                             |
|------------|--------------------------------------------------|
|  Jarius    |  Project Setup, Data Pre-Processing & Feature Engineering
|  Mostafa   |  Energy Prediction ML Model + GUI Integration (Streamlit)
|  Oluwafemi |  NLP/ Prompt Analysis
|  Jatinder  |  Energy Prediction ML Model + GUI integration (Streamlit)


#### Methodology:

4. To accomplish our goal, our methodology involved dataset construction, feature engineering, model training and interface development to create a system capable of estimating and visualizing prompt level energy usage. We needed to be able to have prompts that also have energy readings to be able to train the model so it could understand the relationship between prompt complexity and energy usage. We then needed to be able to visualize the data to interpret how well our models are performing and what we can do to better tune them.






4.1. Data Source and Synthetic Energy Labels

- We imported a hugging face dataset that contains text from yelp reviews, which was the base for generating optimized version of prompts as well as understanding how much energy consumption a prompt would take. While the prompts were real, the energy metrics as well as other metrics were not available. To fix this, we designed a formula to produced synthetic but meaningful targets. The generated labels allow us to create a controlled environment for testing and how text complexity could influence predicted resource usage.

- Just because the energy readings are synthetic does not make the project void as well. The program is designed to be able to just simply swap out datasets and if the energy reading is provided within the dataset it would be able to read it and the models would be trained based on that new data with accurate energy readings.



In [ ]:
#Load dataset

def load_dataset_raw():
    os.makedirs("../data/raw", exist_ok=True)
    #Load yelp review (full train split)
    dataset = load_dataset('yelp_review_full' ,split="train[:100000]") #Limit to 100,000 instead of full 650,000 for much better performance and stability

    print(f"Loaded full dataset with {len(dataset)} rows.")
    #create a df with only prompt text
    df = pd.DataFrame({"prompt": [x["text"] for x in dataset]})
    df = df.rename(columns={"text": "prompt"})

    #save raw data to CSV
    df.to_csv("data/raw/raw_prompts.csv", index=False)
    return df

In [ ]:
#Snippet of energy usage calculation

#Creating synthetic data based on raw dataset
def generate_energy_data(df):
    #Generate synthetic energy labels for testing
    df = df.copy()
    df["energy_kwh"] = (
        0.5 #baseline energy cost
        + df["token_count"] * 0.003 #increass energy proportionally with number of tokens in prompt
        + df["avg_word_length"] * 0.10 # slightly increases energy prompt with longer words
        + df ["token_count"] * df["avg_word_length"] * 0.001
        + (df["token_count"] ** 2) * 0.00005
        + np.random.normal(0,0.2, size=len(df)) #small noise to set to make dataset more realistic and avoid perfect linearity
    )
    df.to_csv("data/synthetic/energy_dataset.csv", index=False)
    return df

4.2  Feature Engineering
- Each prompt was processed to extract linguistic and structural characteristics like word count, token count and average word length. These features were chosen because they all reflect prompt complexity

- By quantifying these aspects we provide the model with signals that could help it learn the relationship between text structure and the energy metrics

In [ ]:
#Snippet of the feature engineering
from src.utils.feature_engineering import tokenizer, stop_words

def compute_feature(prompt, num_layers, training_hours, flops_per_hour):
    #Compute features for a given prompt and model params
    word = prompt.split()
    chars = len(prompt)
    #token count using tokenizer #truncate to avoid maximum sequence index error so that all 650,000 rows are processed
    token_counter = len(tokenizer.encode(prompt, truncation=True, max_length=512))
    #ratio of punctiuatioin characters
    punct_ratio = sum(1 for c in prompt if c in ".,!?;:") /max(chars,1)
    #Average word length
    avg_word_len = sum(len(w) for w in word) /max(len(word),1)
    stopword_ratio = sum(1 for w in word if w.lower() in stop_words) /max(len(word),1)
    #Derived numeric features
    flops_per_layer = flops_per_hour / max(num_layers,1)
    training_efficiency = training_hours / max(num_layers,1)

4.3 Model Selection & Training
- We chose two suprivced learning approaches
    - Linear Regression:
        - The model provides a simple interpretable baseline that helps reveal whether the relationship between text features and synthetic labels ir roughly linear. It's useful for understanding feature influence even if it is more complex
    - Random Forrest
        - This was selected because it handles the relationships and feature interactions effectively. Since prompt complexity can change energy usage in even little ways, Random Forest provides flexibility and typically would give higher accuracy predictions which made it a good choice for the project
- Both projects were trained on the same dataset and we included both models to allow us to compare performance and choose the most suitable predictor for the application

In [ ]:
 #Snippet of model selection
 if model_type == "RandomForest":
            self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        elif model_type == "LinearRegression":
            self.model = LinearRegression()
        else:
            raise ValueError("model_type must be 'RandomForest' or 'LinearRegression'")


4.4 Visualization & Interpretation
- To support our work, we designed plots that would compare the model's predicted values vs the actual energy values. These plots can be seen on the main page of the application after entering the prompt
- These visualizations can help both users and developers understand the model accuracy, identify biases and asses what additional features or adjustments are needed

In [ ]:
#Snippet of plot for actual and predicted  for eneergy consumption
        fig, ax = plt.subplots(figsize=(10, 6))

        ax.plot(x, y_real, label="Actual", color="#1f77b4", marker="o")
        ax.plot(x, preds_real, label="Predicted", color="#ff7f0e",
                linestyle="--", marker="x")


4.5 Prompt Optimization

- To encourage more efficient prompt engineering, the system also includes an optimization feature. When activated, it generates a rewritten version of the user's prompt that retains the original meaning but reduces complexity. This prompts awareness of how phrasing impacts energy usafe and helps users learn how to create cleaner, concise prompts


In [ ]:
 def optimize(
        self,
        prompt: str,
        max_length: int = 128,
        num_beams: int = 4,
        temperature: float = 0.7,
        do_sample: bool = False
    ) -> Tuple[str, List[str]]:

        if not prompt or not prompt.strip():
            return prompt, []

        changes_made = []


        rule_optimized, rule_changes = self._fallback_optimize(prompt)
        changes_made.extend(rule_changes)

        orig_tokens = self._count_tokens(prompt)
        rule_tokens = self._count_tokens(rule_optimized)

        # If rule-based already achieved significant reduction, use it
        rule_reduction = (orig_tokens - rule_tokens) / max(orig_tokens, 1)

#### System Architecture:





5 . Data Input and Pre Processing Layer
- The system loads the text from dataset. The text is passed to the pre processing module which cleans the data by normalizing and performs tokenization, counts words and extraction of text complexity metrics

In [ ]:
  #Load data snippet
   dataset = load_dataset('yelp_review_full' ,split="train[:100000]")

    print(f"Loaded full dataset with {len(dataset)} rows.")

    df = pd.DataFrame({"prompt": [x["text"] for x in dataset]})
    df = df.rename(columns={"text": "prompt"})



  #Tokken Snippet
  tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")



5.1 Feature Engineering
- The extracted test metrics (word count, token count, average word length) are compiled and added as features to the dataset.

In [ ]:
def compute_feature(prompt, num_layers, training_hours, flops_per_hour):
    #Compute features for a given prompt and model params

    word = prompt.split()
    chars = len(prompt)

    token_counter = len(tokenizer.encode(prompt, truncation=True, max_length=512))

    #ratio of punctiuatioin characters
    punct_ratio = sum(1 for c in prompt if c in ".,!?;:") /max(chars,1)

    #Average word length
    avg_word_len = sum(len(w) for w in word) /max(len(word),1)


    #Ratio of stopwrods

    stopword_ratio = sum(1 for w in word if w.lower() in stop_words) /max(len(word),1)

    #Derived numeric features
    flops_per_layer = flops_per_hour / max(num_layers,1)
    training_efficiency = training_hours / max(num_layers,1)


5.2 Synthetic Energy Label
- A custom formula assigns synthetic values for energy consumption. This labels long with FLOPs per hour and training efficiency form the target values for model training and a synthetic dataset is saved to be used for training

In [ ]:
 df = df.copy()
    df["energy_kwh"] = (
        0.5 #baseline energy cost
        + df["token_count"] * 0.003 #increass energy proportionally with number of tokens in prompt
        + df["avg_word_length"] * 0.10 # slightly increases energy prompt with longer words
        + df ["token_count"] * df["avg_word_length"] * 0.001
        + (df["token_count"] ** 2) * 0.00005
        + np.random.normal(0,0.2, size=len(df)) #small noise to set to make dataset more realistic and avoid perfect linearity
    )
    df.to_csv("data/synthetic/energy_dataset.csv", index=False)

5.3 Machine Learning Model
- Two models are implemented to take the engineered features & text data and are trained to learn the relationship between text prompt complexity and energy usage

In [2]:
#Model selection Snippet
     if model_type == "RandomForest":
            self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        elif model_type == "LinearRegression":
            self.model = LinearRegression()
        else:


# Train model snippet

 def train(self):
        print(f"--- Starting Training ({self.model_type}) ---")

        (X_train, y_train), (X_test, y_test) = self.preprocess_and_split()

        # Flatten y_train for sklearn
        # ravel is used to convert a multi-dimensional array into a one-dimensional (1D) array.
        self.model.fit(X_train, y_train.ravel())
        self.anomaly_model.fit(X_train)

        self.is_trained = True
        self._evaluate(X_test, y_test)

        print(f"Training complete. R2 Score: {self.metrics['r2']:.4f}")


IndentationError: unindent does not match any outer indentation level (<string>, line 3)

5.4 Prediction & Visualization
- When a user enters a prompt the system processes the text, extracts the features, runs the trained model to predict the energy usage and then generates plots comparing the predicted and actual values. The metrics are all displayed with summaries and plots on the UI

In [ ]:
#Prediction Snippet
    def estimate(self, prompt_text, layers, training_hours, flops_str):
        # 1. Parse Inputs
        token_count = ComplexityAnalyzer.get_token_count(prompt_text)

        try:
            flops_val = float(flops_str)
            flops_score = (flops_val / 1e18) * 100
            if flops_score < 10: flops_score = 10
        except ValueError:
            flops_score = 10.0

        input_data = pd.DataFrame({
            'num_layers': [layers],
            'training_hours': [training_hours],
            'flops_per_hour': [flops_score],
            'token_count': [token_count],
            'avg_word_length': [ComplexityAnalyzer.get_avg_word_length(prompt_text)]
        })

        # Get raw prediction
        param = input_data.iloc[0].to_dict()
        predicted_energy = self.predict_energy(param)
        predicted_energy = max(0.1, predicted_energy)

        suggestion = "✅ Optimized."
        if predicted_energy > 50:
            suggestion = "⚠️ High Consumption. Consider reducing layers."
        elif token_count > 1000:
            suggestion = "⚠️ Context too long. Summarize input."

        return {
            "energy_kwh": round(predicted_energy, 4),
            "carbon_kg": round(predicted_energy * 0.475, 4),
            "suggestion": suggestion,
            "token_count": token_count,
            "predicted_val": predicted_energy
        }


#Plot pred vs actual snippet
  # Axis settings
    ax.set_title(f"Energy Prediction vs Actual ({self.model_type})")
    ax.set_xlabel("Number of Layers")
    ax.set_ylabel("Energy Consumption (kWh)")
    ax.grid(True, alpha=0.4)
    ax.legend()

5.5 Training Optimization
- The optimizer is trained based on different metrics like semantics score or quality and is then evaluated on its performance

In [ ]:
# Evaluate model snippet
def evaluate_model(optimizer: T5PromptOptimizer, test_prompts: list) -> dict:

    print("\n" + "=" * 60)
    print("Evaluating Model Performance")
    print("=" * 60)

    similarity_checker = SemanticSimilarity()
    validator = EnhancedPromptValidator()

    results = {
        "total_prompts": len(test_prompts),
        "avg_token_reduction": 0,
        "avg_semantic_similarity": 0,
        "avg_quality_score": 0,
        "meaning_preserved_count": 0,
        "by_category": {}
    }

    total_token_reduction = 0
    total_similarity = 0
    total_quality = 0

    for i, test in enumerate(test_prompts):
        original = test.get('original', '')
        expected = test.get('optimized', '')
        category = test.get('category', 'general')

        # Get optimization
        opt_result = optimizer.get_full_optimization(original)

        # Validate
        validation = validator.validate_prompt_optimization(original, opt_result.optimized_prompt)

        # Track metrics
        total_token_reduction += opt_result.token_reduction
        total_similarity += validation['semantic_similarity']
        total_quality += validation['quality_score']

        if validation['is_valid']:
            results['meaning_preserved_count'] += 1

        # Track by category
        if category not in results['by_category']:
            results['by_category'][category] = {
                'count': 0,
                'avg_reduction': 0,
                'avg_similarity': 0
            }
        results['by_category'][category]['count'] += 1

        # Print sample results (first 5)
        if i < 5:
            print(f"\nTest {i+1}:")
            print(f"  Original: {original[:80]}...")
            print(f"  Optimized: {opt_result.optimized_prompt}")
            print(f"  Expected: {expected}")
            print(f"  Token Reduction: {opt_result.token_reduction}%")
            print(f"  Similarity: {validation['semantic_similarity']:.2%}")

    # Calculate averages
    n = len(test_prompts)
    results['avg_token_reduction'] = round(total_token_reduction / n, 1)
    results['avg_semantic_similarity'] = round(total_similarity / n, 3)
    results['avg_quality_score'] = round(total_quality / n, 3)
    results['meaning_preservation_rate'] = round(results['meaning_preserved_count'] / n * 100, 1)

    return results

5.6 Prompt Optimizer
- The T5 model is combined with rule-based transformation to rephrase prompts  to be more energy-efficient while minting semantic meaning. If the user selects the optimization option, the system rewrites the prompt and the optimized prompt is re-evaluated and presented with updated predictions in an attempt to lower energy consumption


In [ ]:
 #Optimizer Snipet
 def get_full_optimization(self, prompt: str) -> OptimizationResult:
        """
        Get complete optimization result with all metrics.

        Args:
            prompt: Original prompt to optimize

        Returns:
            OptimizationResult with all optimization metrics
        """
        original_tokens = self._count_tokens(prompt)

        # Optimize (now returns tuple)
        optimized, changes_made = self.optimize(prompt)
        optimized_tokens = self._count_tokens(optimized)

        # Calculate metrics
        token_reduction = 1 - (optimized_tokens / max(original_tokens, 1))
        token_reduction = max(0, token_reduction)  # Ensure non-negative

        # Estimate energy reduction (proportional to token reduction)
        # Based on research: energy ~ O(n^2) for transformer attention
        energy_reduction = 1 - ((optimized_tokens / max(original_tokens, 1)) ** 2)
        energy_reduction = max(0, min(1, energy_reduction))

        # Semantic similarity
        semantic_similarity = self._estimate_semantic_similarity(prompt, optimized)

        # Quality rating based on both reduction AND semantic preservation
        if token_reduction > 0.4 and semantic_similarity > 0.7:
            quality = "Excellent"
        elif token_reduction > 0.25 and semantic_similarity > 0.6:
            quality = "Good"
        elif token_reduction > 0.1 and semantic_similarity > 0.5:
            quality = "Moderate"
        elif token_reduction > 0:
            quality = "Minimal"
        else:
            quality = "No Change"

        return OptimizationResult(
            original_prompt=prompt,
            optimized_prompt=optimized,
            original_tokens=original_tokens,
            optimized_tokens=optimized_tokens,
            token_reduction=round(token_reduction * 100, 1),
            semantic_similarity=round(semantic_similarity * 100, 1),
            energy_reduction_estimate=round(energy_reduction * 100, 1),
            optimization_quality=quality,
            changes_made=changes_made
        )

#prompt optimize snippet
def optimize_prompt(prompt: str) -> str:
    """
    Quick function to optimize a single prompt.

    Args:
        prompt: The prompt to optimize

    Returns:
        Optimized prompt string
    """
    optimizer = T5PromptOptimizer()
    optimized, _ = optimizer.optimize(prompt)
    return optimized

5.7 User Interface (Streamlit)
- Streamlit serves as the front-end environment where
    - The user enters the prompt
    - THe predictions and visualizations are displayed
    - Other metrics can be viewed
    - Optimization can be applied
    - All visualizations are updated interactively

In [ ]:
#Streamlit GUI Snippet

# -------------------------------------
# PROMPT INPUT VIEW
# -------------------------------------
def input_prompt_view():
    st.subheader("Enter Prompt Context")
    return st.text_area("Input Prompt:", height=100, placeholder="Enter your prompt here...")

# -------------------------------------
# INITIAL ANALYSIS METRICS
# -------------------------------------
def render_initial_analysis(original, optimization):
    st.divider()
    st.subheader("📊 Analysis Report")

    col1, col2, col3, col4 = st.columns(4)
    col1.metric("Predicted Energy", f"{original['energy_kwh']} kWh")
    col2.metric("Carbon Footprint", f"{original['carbon_kg']} kgCO2")
    col3.metric("Original Tokens", optimization.get('original_tokens', original['token_count']))
    col4.metric("Optimized Tokens", optimization.get('optimized_tokens', '-'))

    st.info("**Message: ✅ Initial analysis complete.**")

# -------------------------------------
# TRAINING PERFORMANCE PLOT
# -------------------------------------
def render_training_plot(estimator, layers):
    st.markdown("#### Initial Model Performance")
    with st.expander("📈 View Energy Performance Graph", expanded=True):
        fig = estimator.get_training_plot(layers)
        st.pyplot(fig)

#### Challenges:
- Some challenges we faced along the way include:

    - Finding the right amount of data to use. (Too much and the model took too long to train and predict, Too little and the model couldn't predict well or optimize properly).

    - Applying propper optimization to prompts. (Sometimes the output of a prompt wouldn't resemble the original prompt at all).

    - Being able to properly visualize the prediction label on streamlit gui. (There were issues with being able to plot the labels that were needed for the plots while using streamlit).


### Conclusion:
- In conclusion, we were able to effectively collaborate and create a working program that takes the users prompts outputs a multitude of metrics including energy consumption, plots it on a graph to see the predicted vs actual, and then allow the user to optimize their prompt to help promote sustainability in AI.

- The next step that could be taken to continue to bring the program to the next level is being able to apply the optimizer to the chatbot itself to process a prompt, optimize it and then give it to the chat bot to then consume less energy then it would have before